In [9]:
print('pip install HSE')
# !pip install pandas
# !pip install pyarrow
# !pip install tqdm

pip install HSE


In [5]:
import pandas as pd
import pyarrow.parquet as pq
import datetime
import time
import tqdm
from threading import Thread
import re
import os

def parquet_to_csv(paths): # "./data/RowData/player_log/player=1547/month=2020-11.parquet"
    for path in paths:
        folder = re.sub(os.path.basename(path), '', path)
        filename = re.sub('parquet', 'csv', os.path.basename(path))
        df = pq.read_table(source=path).to_pandas()
        file = open(f"{folder}/{filename}", 'w')
        file.write('month,week_day,half_hour,efficiency,unix,views\n')
        for i in tqdm.tqdm(range(max(df.shape[0], 10))):
#         for i in range(max(df.shape[0], 10)):
            obj = df.loc[i]
            in_t = datetime.datetime.fromtimestamp(obj['DateTimeInTick'] / 1000)
            out_t = datetime.datetime.fromtimestamp(obj['DateTimeOutTick'] / 1000)
            month_1 = in_t.month
            month_2 = out_t.month
            weekday_1 = in_t.weekday()
            weekday_2 = out_t.weekday()
            halfhour_1 = 2 * in_t.hour + 1 + int(in_t.minute >= 30)
            halfhour_2 = 2 * in_t.hour + 1 + int(in_t.minute >= 30)
            efficiency = obj['MacCountAll'] / obj['Duration']
            file.write(f"{month_1},{weekday_1},{halfhour_1},{round(efficiency, 2)},{obj['DateTimeOutTick']},{obj['MacCountAll']}\n")
            if month_1 != month_2 or weekday_1 != weekday_2 or halfhour_1 != halfhour_2:
                file.write(f"{month_2},{weekday_2},{halfhour_2},{round(efficiency, 2)},{obj['DateTimeOutTick']},{obj['MacCountAll']}\n")
        file.close()
        os.rename(f"{folder}/{filename}", f"{folder}/success_{filename}")
# parquet_to_csv(["./data/RowData/player_log/player=1547/month=2020-11.parquet"])
# th1 = Thread(target=parquet_to_csv, args=([
#     "./data/RowData/player_log/player=1547/month=2020-11.parquet",
# ], ))

# th2 = Thread(target=parquet_to_csv, args=([
#     "./data/RowData/player_log/player=1547/month=2020-12.parquet",
# ], ))
# # th3 = Thread(target=parquet_to_csv, args=([
# #     "./data/RowData/player_log/player=1547/month=2020-11.parquet",
# # ], ))

# # th4 = Thread(target=parquet_to_csv, args=([
# #     "./data/RowData/player_log/player=1547/month=2020-12.parquet",
# # ], ))
# th1.start()
# th2.start()

In [ ]:
parquet_to_csv([
    "./data/RowData/player_log/player=1547/month=2021-01.parquet",
    "./data/RowData/player_log/player=1547/month=2021-02.parquet"
])

In [169]:
import re
import os

def getConstructionId(path):
    num = re.search(r'player=\d*', path)
    return int(num[0][7:])

def createDistances(path):
    center_coords = 55.030167631283575, 82.92179748091942
    longest_coords = 54.80053466086988, 83.11027315692883

    num = getConstructionId(path)
    data = pd.read_csv('data/player_details_fixed.csv')
    lat = data[data['PlayerId'] == num].GeoLat.values
    if len(lat) > 0:
        lat = float(lat[0])
    else:
        lat = 0
    long = data[data['PlayerId'] == num].GeoLon.values
    if len(long) > 0:
        long = float(long[0])
    else:
        long = 0
    
    construction_coords = (lat, long)

    if construction_coords != (0, 0):
        distance = round(1 - math.dist(center_coords, construction_coords), 2)
    else:
        distance = 0.5
        
    with open(path) as old_file:
        items = old_file.read().split('\n')
        
    folder = re.sub(os.path.basename(path), '', path)
    filename = re.sub('success_', 'dist_', os.path.basename(path))
    print(filename)
    new_file = open(f"{folder}/{filename}", 'w')
        
    for i, strr in enumerate(items[:-1]):
        if i == 0:
            strr += ',distances\n' 
        else:
            strr += f',{distance}\n'
        new_file.write(strr)  

createDistances('data/RowData/player_log/player=257/success_month=2020-11.csv')
createDistances('data/RowData/player_log/player=257/success_month=2020-12.csv')
createDistances('data/RowData/player_log/player=257/success_month=2021-1.csv')
createDistances('data/RowData/player_log/player=257/success_month=2021-2.csv')
createDistances('data/RowData/player_log/player=257/success_month=2021-3.csv')
createDistances('data/RowData/player_log/player=257/success_month=2021-4.csv')
createDistances('data/RowData/player_log/player=257/success_month=2021-5.csv')
createDistances('data/RowData/player_log/player=257/success_month=2021-6.csv')

dist_month=2020-11.csv
dist_month=2020-12.csv
dist_month=2021-1.csv
dist_month=2021-2.csv
dist_month=2021-3.csv
dist_month=2021-4.csv
dist_month=2021-5.csv
dist_month=2021-6.csv


In [171]:
import os
data_path = '/root/data/RowData/player_log'
players = os.listdir(data_path)
csv_files = []
# for player in players:
for player in ['player=257']:
    files = os.listdir(f"{data_path}/{player}")
    for file in files:
        if file.startswith('dist') and file.endswith('.csv'):
            csv_files.append(f"{data_path}/{player}/{file}")
# one_thread = len(parquets) // 5
# for i, parquet in enumerate(parquets):
#     print(f'"{parquet}",')
#     if (i + 1) % one_thread == 0:
#         print()
#         print()
# csv_files = csv_files[:2]
print(csv_files)


['/root/data/RowData/player_log/player=257/dist_month=2021-2.csv', '/root/data/RowData/player_log/player=257/dist_month=2021-6.csv', '/root/data/RowData/player_log/player=257/dist_month=2020-11.csv', '/root/data/RowData/player_log/player=257/dist_month=2021-4.csv', '/root/data/RowData/player_log/player=257/dist_month=2020-12.csv', '/root/data/RowData/player_log/player=257/dist_month=2021-5.csv', '/root/data/RowData/player_log/player=257/dist_month=2021-3.csv', '/root/data/RowData/player_log/player=257/dist_month=2021-1.csv']


In [172]:
sum_file = open('data/RowData/player_log/player=257/sum_dist_month.csv', 'w')

with open(csv_files[0]) as first_file:
    headers = first_file.readline()
    sum_file.write(headers)
for csv_file in csv_files:
    with open(csv_file) as file:
        lines = file.readlines()[1:]
        for line in lines:
            sum_file.write(line)
sum_file.close()
#     print(first_line[-1])
#     shift = len(first_line)

#     for i, file in enumerate(csv_files):
#         append_data = open(file).read()[56:]
#         sum_file.write(append_data) 

In [175]:
print
sum_file = open('data/RowData/player_log/player=257/success_month=2020-11.csv', 'r')
lines = sum_file.readlines()
for i, line in enumerate(lines):
    if len(line.split(',')) != 6:
        print('success', i)
        
sum_file = open('data/RowData/player_log/player=257/dist_month=2020-11.csv', 'r')
lines = sum_file.readlines()
for i, line in enumerate(lines):
    if len(line.split(',')) != 7:
        print('dist', i)
        
sum_file = open('data/RowData/player_log/player=257/sum_dist_month.csv', 'r')
lines = sum_file.readlines()
for i, line in enumerate(lines):
    if len(line.split(',')) != 7:
        print('sum', i)
# for line in lines[:11]:
#     print(f"{len(line.split(','))}, {line}")
# for line in lines[464390:464410]:
#     print(f"{len(line.split(','))}, {line}")

In [178]:
data = pd.read_csv('data/RowData/player_log/player=257/sum_dist_month.csv')

1    596026
2    587866
4    581175
6    578982
5    569985
3    565279
0    565174
Name: week_day, dtype: int64

In [180]:
data

,month,week_day,half_hour,efficiency,unix,views,distances
0,2,0,10,2.2,1612144793993,11,0.99
1,2,0,10,1.8,1612144788977,9,0.99
2,2,0,10,2.2,1612144783973,11,0.99
3,2,0,10,1.2,1612144778970,6,0.99
4,2,0,10,2.2,1612144773957,11,0.99
...,...,...,...,...,...,...,...
4044482,2,0,3,0.2,1612130630600,1,0.99
4044483,2,0,3,0.6,1612130625587,3,0.99
4044484,2,0,3,0.6,1612130620580,3,0.99
4044485,2,0,3,0.4,1612130615577,2,0.99


In [ ]:
def get_k_notes(weekday):
    data

In [179]:
data['week_day'].value_counts()

1    596026
2    587866
4    581175
6    578982
5    569985
3    565279
0    565174
Name: week_day, dtype: int64

In [20]:
import datetime
import tqdm

sum_file = open('data/RowData/player_log/player=257/sum_dist_month.csv', 'r')
lines = sum_file.readlines()
lines = lines[1:]

sum_dict = {}

# for line in lines[:600]:
for line in tqdm.tqdm(lines):
#     print(f"{line}", end='')
    ticks = int(line.split(',')[4])
    dt = datetime.date.fromtimestamp(ticks / 1000)
    halfhour = line.split(',')[2]
    weekday = line.split(',')[1]
    views = int(line.split(',')[5])
    dict_key = str(dt) + f'-{weekday}' + f'-{halfhour}'
    if sum_dict.get(dict_key) is None:
        sum_dict[dict_key] = views
    else:
        sum_dict[dict_key] += views

lesser_dict = {}
for s in tqdm.tqdm(sum_dict.items()):
    key_parts = s[0].split('-')
    new_key = key_parts[3] + '-' + key_parts[4]
    
    value = s[1]
    
    if lesser_dict.get(new_key) is None:
        lesser_dict[new_key] = [value, 1]
    else:
        lesser_dict[new_key][0] += value
        lesser_dict[new_key][1] += 1
        
len(sum_dict), len(lesser_dict)

100%|██████████| 11599/11599 [00:00<00:00, 90128.65it/s]


(11599, 336)

In [11]:
lesser_dict

{'0-10': [135833, 34],
 '0-9': [118166, 34],
 '0-8': [94847, 34],
 '0-12': [132658, 33],
 '0-11': [128787, 34],
 '0-14': [136180, 33],
 '0-13': [131299, 33],
 '0-16': [143957, 32],
 '0-15': [136646, 32],
 '0-18': [145687, 32],
 '0-17': [148099, 32],
 '0-20': [152554, 32],
 '0-19': [152884, 32],
 '0-24': [161113, 32],
 '0-23': [153380, 32],
 '0-22': [149683, 32],
 '0-26': [170821, 32],
 '0-25': [157926, 32],
 '0-30': [183424, 31],
 '0-29': [190043, 31],
 '0-28': [181746, 32],
 '0-32': [150530, 32],
 '0-31': [163339, 31],
 '0-38': [108803, 35],
 '0-37': [121784, 35],
 '0-36': [128223, 35],
 '0-42': [65766, 34],
 '0-41': [68047, 34],
 '0-40': [84504, 35],
 '0-44': [52553, 35],
 '0-43': [55784, 34],
 '1-2': [42878, 35],
 '1-1': [43211, 35],
 '0-48': [42780, 70],
 '1-48': [40456, 70],
 '0-7': [70057, 32],
 '0-6': [58448, 34],
 '0-21': [151902, 32],
 '0-27': [176370, 32],
 '0-34': [137584, 32],
 '0-33': [149114, 32],
 '0-35': [125632, 33],
 '0-39': [97606, 35],
 '0-46': [48439, 35],
 '0-45':

In [37]:
sum_file = open('data/RowData/player_log/player=257/sum_dist_month.csv', 'r')
headers = sum_file.readline()
lines = sum_file.readlines()

new_file = open('data/RowData/player_log/player=257/mean_sum_dist_month.csv', 'w')
new_file.write(headers.strip() + ',mean_half_hour\n')

for line in tqdm.tqdm(lines):
    halfhour = line.split(',')[2]
    weekday = line.split(',')[1]
    key = f'{weekday}-{halfhour}'
    aggregate_values = lesser_dict.get(key)
    new_file.write(line.strip() + f',{round(aggregate_values[0] / aggregate_values[1], 2)}\n')
new_file.close()

month,week_day,half_hour,efficiency,unix,views,distances

month,week_day,half_hour,efficiency,unix,views,distances,mean_half_hour


100%|██████████| 4044487/4044487 [00:25<00:00, 159025.60it/s]


In [40]:
sum_file = open('data/RowData/player_log/player=257/sum_dist_month.csv', 'r')
headers = sum_file.readline()
lines = sum_file.readlines()

weekday_dict = {}

for line in tqdm.tqdm(lines):
#     print(f"{line}", end='')
    ticks = int(line.split(',')[4])
    dt = datetime.date.fromtimestamp(ticks / 1000)
    halfhour = line.split(',')[2]
    weekday = line.split(',')[1]
    views = int(line.split(',')[5])
    
    dict_key = str(weekday)
    if weekday_dict.get(dict_key) is None:
        weekday_dict[dict_key] = [views, 1]
    else:
        weekday_dict[dict_key][0] += views
        weekday_dict[dict_key][1] += 1

100%|██████████| 4044487/4044487 [00:30<00:00, 133593.90it/s]


In [42]:
weekday_dict

{'0': [5475212, 565174],
 '1': [6307783, 596026],
 '2': [6385424, 587866],
 '3': [6280730, 565279],
 '4': [6935711, 581175],
 '5': [5623675, 569985],
 '6': [4837617, 578982]}

In [41]:
sum_file = open('data/RowData/player_log/player=257/mean_sum_dist_month.csv', 'r')
headers = sum_file.readline()
lines = sum_file.readlines()

new_file = open('data/RowData/player_log/player=257/new_mean_sum_dist_month.csv', 'w')
new_file.write(headers.strip() + ',mean_weekday\n')

for line in tqdm.tqdm(lines):
    weekday = line.split(',')[1]
    key = str(weekday)
    weekday_values = weekday_dict.get(key)
    new_file.write(line.strip() + f',{round(weekday_values[0] / weekday_values[1], 2)}\n')
new_file.close()

100%|██████████| 4044487/4044487 [00:27<00:00, 149017.30it/s]


In [13]:
!pip install requests
import requests
response = requests.get('http://api.openweathermap.org/data/2.5/weather?lat=55.04&lon=82.91&appid=4ab81a655f265a02206fd1c134001de4').json()
if response['rain']:
    




You should consider upgrading via the '/root/venv/bin/python3.9 -m pip install --upgrade pip' command.


KeyError: 'rain'

In [45]:
import tqdm
import datetime
# params - list of numbers(columns indexes in file)
# date_info - 'None', 'day', 'month', 'year', or 'full' as optional features
def create_dict(params, lines, date_info=None, min_lines=0, max_lines=500000):
    new_dict = {}
    specials = []
    for line in tqdm.tqdm(lines[min_lines:max_lines]):
    #     print(f"{line}", end='')
        note_params = []
        for i in params:
            if i == 4:
                if date_info is not None:
                    dt = datetime.date.fromtimestamp(int(line.split(',')[i]) / 1000)
                    if date_info == 'day' or date_info == 'full':
                        note_params.append(dt.day)
                    if date_info == 'month' or date_info == 'full':
                        note_params.append(dt.month)
                    if date_info == 'year' or date_info == 'full':
                        note_params.append(dt.year)
            elif '.' in line.split(',')[i]:
                note_params.append(float(line.split(',')[i]))
            else:
                note_params.append(int(line.split(',')[i]))
        
        views = int(line.split(',')[5])

        dict_key = str(note_params[0])
        for p in note_params[1:]:
            dict_key += f'-{p}'
            
        if new_dict.get(dict_key) is None:
            new_dict[dict_key] = [views, 1]
        else:
            new_dict[dict_key][0] += views
            new_dict[dict_key][1] += 1
            
    return new_dict
            
sum_file = open('data/RowData/player_log/player=257/mean_sum_dist_month.csv', 'r')
headers = sum_file.readline()
lines = sum_file.readlines()

create_dict([4], lines, 'full')

100%|██████████| 500000/500000 [00:04<00:00, 106474.97it/s]


{'1-2-2021': [182073, 15091],
 '2-2-2021': [137256, 17249],
 '3-2-2021': [200607, 17259],
 '4-2-2021': [213284, 17249],
 '5-2-2021': [160326, 17250],
 '6-2-2021': [183091, 16530],
 '7-2-2021': [179980, 16532],
 '8-2-2021': [209798, 17249],
 '9-2-2021': [36536, 17248],
 '10-2-2021': [169061, 17248],
 '11-2-2021': [209236, 17251],
 '12-2-2021': [216724, 17248],
 '13-2-2021': [193214, 17252],
 '14-2-2021': [177779, 17248],
 '15-2-2021': [217918, 17250],
 '16-2-2021': [211624, 17251],
 '17-2-2021': [196305, 17249],
 '18-2-2021': [131384, 17250],
 '19-2-2021': [190722, 17249],
 '20-2-2021': [220794, 17248],
 '21-2-2021': [106086, 17250],
 '22-2-2021': [157234, 17251],
 '23-2-2021': [140959, 17248],
 '24-2-2021': [163242, 16183],
 '25-2-2021': [5063, 1163],
 '26-2-2021': [117699, 17248],
 '27-2-2021': [172657, 17250],
 '28-2-2021': [145495, 17249],
 '1-3-2021': [11118, 2156],
 '1-6-2021': [263785, 15090],
 '2-6-2021': [257666, 17250],
 '3-6-2021': [23684, 3261]}

In [5]:
import pandas as pd
import pyarrow.parquet as pq
import datetime

In [6]:
df = pq.read_table(source="./data/RowData/player_log/player=1547/month=2020-11.parquet").to_pandas()

In [8]:
print(df.shape)
new_df = pd.DataFrame(columns=('month', 'week_day', 'half_hour', 'efficiency', 'unix', 'views'), )
new_df

(494703, 14)


,month,week_day,half_hour,efficiency,unix,views


In [6]:
parquet_to_csv([
    "./data/RowData/player_log/player=1547/month=2020-11.parquet",
    "./data/RowData/player_log/player=1547/month=2020-12.parquet"
])

  5%|▌         | 26645/494703 [00:11<03:21, 2324.04it/s]


KeyboardInterrupt: 

In [ ]:
new_df.shape

In [9]:
new_df

,month,week_day,half_hour,efficiency,unix,views


In [ ]:
new_df.to_csv('./preprocessing_data.csv')


In [103]:
import math

center_coords = 55.030167631283575, 82.92179748091942
longest_coords = 54.80053466086988, 83.11027315692883
custom_coords = 54.96091109214436, 82.95228230422312
1 - math.dist(center_coords, custom_coords)
#math.dist(center_coords, custom_coords)
#28, 26 км

0.9243310323487922